In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.sparse import csc_matrix, lil_matrix, save_npz, hstack

In [4]:
ratings1 = pd.read_csv('../ratings.csv')
books_info = pd.read_csv('../books_meta_info.csv')

In [5]:
ratings1.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [6]:
books_info.head()

,Unnamed: 0,book_id,goodreads_book_id,authors,original_publication_year,language_code,Art,Biography,Business,Chick Lit,...,Romance,Science,Science Fiction,Self Help,Spirituality,Sports,Suspense,Thriller,Travel,Young Adult
0,0,1,2767052,Suzanne Collins,2008.0,eng,0.0,0.0,0.0,0.0,...,3341.0,0.0,8772.0,0.0,0.0,0.0,641.0,800.0,0.0,25968.0
1,1,2,3,"J.K. Rowling, Mary GrandPré",1997.0,eng,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14984.0
2,2,3,41865,Stephenie Meyer,2005.0,en-US,0.0,0.0,0.0,716.0,...,0.0,0.0,253.0,0.0,0.0,0.0,0.0,0.0,0.0,16640.0
3,3,4,2657,Harper Lee,1960.0,eng,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2436.0
4,4,5,4671,F. Scott Fitzgerald,1925.0,eng,0.0,0.0,0.0,0.0,...,2090.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,397.0


In [7]:
ratings = ratings1.merge(books_info, on='book_id')

In [2]:
# ratings = pd.read_csv("../ratings_meta_info.csv", index_col=0)

FileNotFoundError: File b'../ratings_meta_info.csv' does not exist

In [8]:
ratings.head()

,user_id,book_id,rating,Unnamed: 0,goodreads_book_id,authors,original_publication_year,language_code,Art,Biography,...,Romance,Science,Science Fiction,Self Help,Spirituality,Sports,Suspense,Thriller,Travel,Young Adult
0,1,258,5,257,1232,"Carlos Ruiz Zafón, Lucia Graves",2001.0,eng,0.0,0.0,...,355.0,0.0,0.0,0.0,0.0,0.0,230.0,311.0,0.0,0.0
1,11,258,3,257,1232,"Carlos Ruiz Zafón, Lucia Graves",2001.0,eng,0.0,0.0,...,355.0,0.0,0.0,0.0,0.0,0.0,230.0,311.0,0.0,0.0
2,143,258,4,257,1232,"Carlos Ruiz Zafón, Lucia Graves",2001.0,eng,0.0,0.0,...,355.0,0.0,0.0,0.0,0.0,0.0,230.0,311.0,0.0,0.0
3,242,258,5,257,1232,"Carlos Ruiz Zafón, Lucia Graves",2001.0,eng,0.0,0.0,...,355.0,0.0,0.0,0.0,0.0,0.0,230.0,311.0,0.0,0.0
4,325,258,4,257,1232,"Carlos Ruiz Zafón, Lucia Graves",2001.0,eng,0.0,0.0,...,355.0,0.0,0.0,0.0,0.0,0.0,230.0,311.0,0.0,0.0


In [9]:
ratings.language_code.value_counts()

eng      4239947
en-US    1179179
en-GB     111743
en-CA      47106
spa        25033
fre        10815
ara        10543
ger         3837
ind         2649
jpn         1190
en          1171
por         1043
nl          1017
nor          963
pol          961
per          700
dan          479
ita          307
mul          195
vie          179
swe          129
fil           75
tur           67
rus           67
rum           64
Name: language_code, dtype: int64

In [10]:
ratings.columns

Index(['user_id', 'book_id', 'rating', 'Unnamed: 0', 'goodreads_book_id',
       'authors', 'original_publication_year', 'language_code', 'Art',
       'Biography', 'Business', 'Chick Lit', 'Children's', 'Christian',
       'Classics', 'Comics', 'Contemporary', 'Cookbooks', 'Crime', 'Ebooks',
       'Fantasy', 'Fiction', 'Gay and Lesbian', 'Graphic Novels',
       'Historical Fiction', 'History', 'Horror', 'Humor and Comedy', 'Manga',
       'Memoir', 'Music', 'Mystery', 'Nonfiction', 'Paranormal', 'Philosophy',
       'Poetry', 'Psychology', 'Religion', 'Romance', 'Science',
       'Science Fiction', 'Self Help', 'Spirituality', 'Sports', 'Suspense',
       'Thriller', 'Travel', 'Young Adult'],
      dtype='object')

In [11]:
len(ratings.user_id.unique())

53424

In [12]:
ratings.user_id.max()

53424

In [13]:
user_ids = ratings.user_id.values

In [14]:
type(user_ids)

numpy.ndarray

In [15]:
book_ids = ratings.book_id.values

In [16]:
y = ratings.rating.values

### book mapping

In [12]:
# book_id_mapping = ratings.drop(['user_id', 'rating'], axis = 1)

In [13]:
# book_id_mapping = book_id_mapping.drop_duplicates()

In [14]:
# mapping = np.arange(10000)

In [15]:
# book_id_mapping['mapping'] = mapping

In [16]:
# book_id_mapping.tail()

,book_id,mapping
5835494,3885,9995
5837194,9569,9996
5837312,9580,9997
5858335,8892,9998
5876842,9548,9999


In [17]:
# book_id_mapping.to_csv('mapping_goodreads_ids.csv')

In [18]:
# ratings_with_map = ratings.merge(book_id_mapping, how = 'left', left_on = 'book_id', right_on = 'book_id')

In [19]:
# ratings_with_map.shape

(5976479, 4)

In [20]:
# book_ids = ratings_with_map.mapping.values

### Converting to numpy one hot

In [17]:
book_ids.size

5976479

In [18]:
def numpy_get_dummies(array):
    b = csc_matrix((array.size, array.max()+1))
#    b = lil_matrix((array.size, array.max()+1))
    print(b.shape)
#    b = np.zeros((array.size, array.max()+1))
#    b = csc_matrix(b)
#     temp = np.arange(array.size)
#     for i in temp:
#         b[i, array[i]] = 1
#    b[temp, array[temp]] = 1
    b[np.arange(array.size), array] = 1
    return b

In [19]:
book_ids_dummies = numpy_get_dummies(book_ids)

(5976479, 10001)


/Users/stephaniedoctor/anaconda/envs/py3k/lib/python3.5/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [20]:
type(book_ids_dummies)

scipy.sparse.csc.csc_matrix

In [21]:
save_npz('book_ids_dummies.npz', book_ids_dummies)

In [21]:
user_ids_dummies = numpy_get_dummies(user_ids)

(5976479, 53425)


/Users/stephaniedoctor/anaconda/envs/py3k/lib/python3.5/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [22]:
user_ids_dummies.sum()

5976479.0

In [24]:
save_npz('user_ids_dummies.npz', user_ids_dummies)

In [25]:
np.savetxt('y.txt', y)

### Concatenating the two

In [23]:
user_ids_dummies.shape

(5976479, 53425)

In [43]:
type(user_ids_dummies)

scipy.sparse.csc.csc_matrix

In [24]:
book_ids_dummies.shape

(5976479, 10001)

In [25]:
def concatenate_csc_matrices_by_columns(matrix1, matrix2):
    new_data = np.concatenate((matrix1.data, matrix2.data))
    new_indices = np.concatenate((matrix1.indices, matrix2.indices))
    new_ind_ptr = matrix2.indptr + len(matrix1.data)
    new_ind_ptr = new_ind_ptr[1:]
    new_ind_ptr = np.concatenate((matrix1.indptr, new_ind_ptr))

    return csc_matrix((new_data, new_indices, new_ind_ptr))

In [26]:
users_books_dummies = concatenate_csc_matrices_by_columns(user_ids_dummies, book_ids_dummies)

In [27]:
users_books_dummies.shape

(5976479, 63426)

In [33]:
save_npz('user_books_dummies.npz', users_books_dummies)

In [28]:
type(users_books_dummies)

scipy.sparse.csc.csc_matrix

In [42]:
users_books_dummies.shape

(5976479, 63426)

In [37]:
nlp = np.array(pd.read_csv('nlp_features.csv', header=None))

In [38]:
nlp = nlp[:, 1:]

In [39]:
nlp.shape

(5976479, 40)

In [40]:
nlp

array([[-0.23059744, -0.25008177, -0.37793511, ..., -0.09215129,
        -1.89263341,  0.21059735],
       [-0.23059744, -0.25008177, -0.37793511, ..., -0.09215129,
        -1.89263341,  0.21059735],
       [-0.23059744, -0.25008177, -0.37793511, ..., -0.09215129,
        -1.89263341,  0.21059735],
       ..., 
       [ 0.1       ,  0.1       ,  0.1       , ...,  0.1       ,
         0.1       ,  0.1       ],
       [ 0.1       ,  0.1       ,  0.1       , ...,  0.1       ,
         0.1       ,  0.1       ],
       [ 0.1       ,  0.1       ,  0.1       , ...,  0.1       ,
         0.1       ,  0.1       ]])

In [44]:
users_books_dummies_nlp = concatenate_csc_matrices_by_columns(users_books_dummies, csc_matrix(nlp))

In [45]:
users_books_dummies_nlp.shape

(5976479, 63466)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(users_books_dummies_nlp, y, test_size=0.2, random_state=42)

In [47]:
from sklearn.datasets import dump_svmlight_file
train_file = 'libfm_train_nlp.txt'
test_file = 'libfm_test_nlp.txt'

In [49]:
with open(train_file, 'wb') as f:
    dump_svmlight_file(X_train, y_train, f=f)

In [50]:
with open(test_file, 'wb') as f:
    dump_svmlight_file(X_test, y_test, f=f)